In [ ]:
indir = snakemake.params.indir
outdir = snakemake.params.outdir
donor = int(snakemake.wildcards.d)
weights = snakemake.params.weights
objectives_l = snakemake.params.get("objectives_l", 
                                    ["variants_with_clone_norm_by_1_over_nclones_with_variant", 
                                     "max_clone_ncells_over_nclones", "max_clone_ncells_over_ncells", 
                                     "pct_thresh","other_pct_thresh", 
                                     "n_vars", "obj_nclones_more_than_one_unique"])
topn = snakemake.params.get("topn", 16)
# try: 
#     af_indirs = snakemake.params.af_indirs
#     indir = snakemake.params.indir
#     outdir = snakemake.params.outdir
#     donor = int(snakemake.wildcards.d)
#     weights = snakemake.params.weights
#     anno_cells_meta_f = snakemake.input.anno_cells_meta_f
#     objectives_l = snakemake.params.get("objectives_l", 
#                                         ["variants_with_clone_norm_by_1_over_nclones_with_variant", 
#                                          "max_clone_ncells_over_nclones", "max_clone_ncells_over_ncells", 
#                                          "pct_thresh","other_pct_thresh", 
#                                          "n_vars", "obj_nclones_more_than_one_unique"])
# except NameError:
#     af_indirs = "/data/Mito_Trace/output/pipeline/v04/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_30/"
#     indir = "/data/Mito_Trace/output/pipeline/v04/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/enriched_barcodes/clones/variants_init/knn/kparam_30/donor0/"
#     outdir = "/data/Mito_Trace/output/pipeline/v04/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/enriched_barcodes/clones/variants_init/knn/kparam_30/donor0/optim_results/"
#     donor = 0
#     anno_cells_meta_f = "/data/Mito_Trace/output/pipeline/v04/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_30/gff_A2_black/annotation_clones/se_cells_meta_labels.tsv"
#     weights = [1,0,0,1,-1, 1, 1]
#     objectives_l = ["variants_with_clone_norm_by_1_over_nclones_with_variant", 
#                     "max_clone_ncells_over_nclones", "max_clone_ncells_over_ncells", 
#                     "pct_thresh","other_pct_thresh", "n_vars", "obj_nclones_more_than_one_unique"] #"nvars"

    
param_names = ["pct_thresh","af_thresh", "other_pct_thresh"]


In [ ]:
## Remove any weights and objectives with 0 
temp_obj = []
temp_weights = []
for ind, o in enumerate(objectives_l):
    if weights[ind] != 0:
        temp_weights.append(weights[ind])
        temp_obj.append(o)
weights = temp_weights
objectives_l = temp_obj
print(objectives_l)
print(weights)

In [ ]:
objectives = {ind:x for ind,x in enumerate(objectives_l)}


In [ ]:
import src.clone_variants_optim  as optim
from mplh import cluster_help as ch

from dynamicTreeCut import cutreeHybrid
from scipy.spatial.distance import pdist
import numpy as np
from scipy.cluster.hierarchy import linkage

import mplh.cluster_help as ch
import mplh.fig_utils as fu

In [ ]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import matplotlib
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from os.path import join, exists, dirname
from glob import glob
import pickle
from icecream import ic
ic.disable()
np.set_printoptions(formatter={'float': lambda x: format(x, '.3f')})
# matplotlib.rcParams['ps.useafm'] = False
# matplotlib.rcParams['pdf.use14corefonts'] = True
# matplotlib.rcParams['text.usetex'] = False

# Load results and get top 10% of them

In [ ]:
results_df = pd.read_csv(join(outdir, "objectives.csv"), index_col=0)
full_params = pd.read_csv(join(outdir, "raw_params.csv"), index_col=0)


## Get the top objective result based on the distribution of scores. 1D cluster scores and take cluster with highest mean. Number of clusters determined using the loss and the elbow plot with kneedle

In [ ]:
#print(results_df.head())
print((results_df>0).any())
drop_inds = results_df.loc[(results_df==0).all(axis=1)].index
results_norm = optim.set_multi(results_df, weights)
rank_df = optim.set_multi_rank(results_norm, weights)

drop_results = results_norm.loc[results_norm["multi"].isnull()]
results_norm = results_norm.loc[~(results_norm["multi"].isnull())]
if len(results_norm) == 0:
    raise ValueError("No param retrieved variants")

########################
## Save the objective results
########################
results_norm.to_csv(join(outdir, "objectives_norm.csv"))
#results_df.to_csv(join(outdir, "objectives.csv"))
#full_params.to_csv(join(outdir, "raw_params.csv"))
rank_df.to_csv(join(outdir, "objectives_rank.csv"))

In [ ]:

#results_norm.loc[results_norm["multi"] == np.nan]

########################
# Plot distribution results
########################
sns.displot(results_norm["multi"])
plt.title("multiobjective function (want to maximize)")
plt.savefig(join(outdir, "loss_multi.pdf"))

sns.displot(rank_df["multi"])

sns.displot(rank_df["variants_with_clone_norm_by_1_over_nclones_with_variant"])
plt.title("objective: variants_with_clone_norm_by_1_over_nclones_with_variant (want to maximize)")
plt.savefig(join(outdir, "loss_variants_with_clone_norm_by_1_over_nclones_with_variant.pdf"))

sns.displot(rank_df["obj_nclones_more_than_one_unique"])
plt.title("objective: Number of clones with at least 2 unique variants (want to maximize)")
plt.savefig(join(outdir, "loss_variants_with_clone_norm_by_1_over_nclones_with_variant.pdf"))


In [ ]:

af_indir = join(indir, "sc_af", f"donor{donor}")

AF_df = pd.read_csv(join(af_indir, "af.tsv"), index_col=0, sep="\t")
DP_df = pd.read_csv(join(af_indir, "dp.tsv"), index_col=0, sep="\t")

print(AF_df.shape)
print(DP_df.shape)
print("Depth")
print(DP_df.head())
AF_df.head()

cells_meta = pd.read_csv(join(indir, "cells_meta.tsv"), sep='\t', index_col="ID")#.sort_values(["donor", "lineage"])
cells_meta["name"] = cells_meta["donor"].astype(str)+"_"+cells_meta["lineage"].astype(str)
curr_labels = cells_meta[cells_meta["donor"]==donor]

conditions = curr_labels["condition"].unique()

## Get donor inds
donor_inds = AF_df.index[((AF_df>0.9).sum(axis=1)>(0.9*AF_df.shape[1]))]


def rm_high(df, thresh, pct_thresh):
    return df.loc[~(((df>thresh).sum(axis=1)>pct_thresh*df.shape[0]))]

def rm_low(df, thresh, pct_thresh):
    return df.loc[~((df<thresh).sum(axis=1)>(pct_thresh*df.shape[1]))]



In [ ]:
########################
# Get the top n results
########################
def get_top_n_results(results_df, rank_df, n=12):
    filt_rank = rank_df.sort_values(by=["multi"])[::-1].iloc[:n]
    filt_results = results_df.loc[filt_rank.index]
    return filt_rank, filt_results


full_params.to_csv(join(outdir, "params.csv"))

filt_rank, filt_results = get_top_n_results(results_norm, rank_df, n=topn)
filt_results.columns = [f"{x}_obj" for x in filt_results.columns]
filt_results = pd.merge(filt_results, full_params, left_index=True, right_index=True, how="left")
filt_rank = filt_rank.loc[filt_results.index]

all_df = []
all_objs = {}
for ind, val in filt_results.iterrows():
    print(ind)
    obj_out, data = optim.evaluate_series(val, AF_df, DP_df, curr_labels, return_data=True)
    all_df.append(data["all_unique_df"])
    all_objs[ind] = obj_out 
all_df = pd.concat(all_df)

heatmap_input = all_df[["n_cells", "variant"]].reset_index().pivot(index="id", columns="variant", values="n_cells").fillna(0).astype(int)
meta_df = all_df[["af_thresh", "other_pct_thresh", "pct_thresh", "clone"]]
meta_df = meta_df.loc[~(meta_df.index.duplicated())]
meta_df = meta_df.sort_values(["af_thresh","pct_thresh", "other_pct_thresh", "clone"])
heatmap_input = heatmap_input.loc[meta_df.index]


In [ ]:
# Get the variants based on total number of cells across parameters
heatmap_input = heatmap_input.loc[:,heatmap_input.sum().sort_values()[::-1].index]
variants_order = heatmap_input.columns
clone_sums = meta_df.groupby("clone").apply(optim.clone_sum, heatmap_input)
clone_sums = clone_sums.loc[:, clone_sums.sum().sort_values()[::-1].index]
clones_order = clone_sums.index

all_df["params"] = all_df.apply(optim.params_to_str, axis=1, args=(param_names,))
filt_results["params"] = filt_results.apply(optim.params_to_str, axis=1, args=(param_names,))
filt_results["params_multi"] = filt_results.apply(optim.params_and_multi_str, axis=1)
tmp = filt_results.set_index("params")
all_df["multi_obj"] = all_df.apply(lambda x: tmp.loc[x["params"], "multi_obj"], axis=1)
del tmp                               

all_df["params_multi"] = all_df.apply(optim.params_and_multi_str, axis=1)


fg = sns.FacetGrid(data=all_df.reset_index(), height=4, sharey=False, sharex=False,
                   col="params", col_wrap=4, col_order=filt_results["params"].values, margin_titles=True)

fg.map_dataframe(optim.draw_heatmap, 'variant','clone', 'log2_n_cells',
                 clones_order=clones_order, variants_order=variants_order)#, cbar=False)

#fg.set_titles(row_template = 'other_pct_thresh: {row_name}', col_template = 'pct_thresh: {col_name}')
fg.fig.suptitle(f"Best parameter combinations shown in order")
fg.fig.subplots_adjust(top=0.9, hspace = 0.8)

plt.title("multiobjective function (want to maximize)")
#plt.savefig(join(outdir, "top_param_results.pdf"))
plt.savefig(join(outdir, "top_param_results.pdf"), dpi=300)

